---

# Code pour les flags 16 à 20



---

# Flag16 : 
rsa.crt distributeur

Il faut demander la clé publique de l'utilisateur huangrobert, ghoward, rebeccataylor sur un terminal de service et extraire les N1,N2,N3 afin d'utiliser ces modulos dans le théorème des restes chinois pour obtenir $m^3$ (car on a $m^{e1}$, $m^{e1}$, $m^{e3}$ dans un document de la bibliothèque : le manuel du distributeur)

Finalement pour obtenir le flag il suffit de se rendre dans la bibliothèque et d'aller au distributeur. Taper ADMIN et rentrer le password obtenu ci-dessous.

In [4]:
from Crypto.Util.number import inverse, long_to_bytes
from Crypto.PublicKey import RSA
from Crypto.PublicKey.RSA import RsaKey

import gmpy2

# Chiffrés donnés par 3 personnes du meme message m (le mot de passe qu'on recherche)
c1 = int("3f4ef2a5afc8adf08840705a5f9071453ecb0f6992f31a6c4d6f78fc752a9b65ff57705235e4f24934f07c0df72e81555c432b9d91bd15d7e4f0b8ba1efaceb55ad811a08a814b5730712053dacce3972cc5d1a98d586367cc0b92979135f25fba4f33fa7f723295a826a725222eda8e7bf78966fa110bc803e11d6a99fa21d55397cd13659a5792a83f8e07d409f75bb1f315336360e6630bfdccf599c5d74a7c94e0cd33de2944e00dc3befbd397165452efa8c8771d13a8101bf2bc3cbb5697d1a1fde79af5a1491d30d1fa7baf746e70e5fa20c29687d1aae5f24b69764e64fd49dd82d69e91b5d812b721580ac633af15733e81e99d38bc91c0dd988cf9", 16)
c2 = int("20a444d170b000e8409f53b5567bd40dd2bc9a121be050cd4486adbff0a140e988fe0116c7608d63dc496641feed7bc7de51f7bc0b21c4055ade8a792411b1ad9a1e5004f4fecdf748dc9e91a050aeb192b154ac6a3f9f87d0841defd6de52fcbc426509aa2c01d01e3db8107032498271b6034ede3b7e57acf5233362813575196ae0c04d8f62ce21151dc5d3e71dbbdee3c6a17d80d4263bd6eaf4906f0efaf34f08f5ee812897e553efa6ab1fcd012cecd7af790650080021f8e1f3134f5054f0f08df23da410521450de9c9c20a8d095c7800131945eb68de428a1960c6b1acaaf1bf3b9837e5fb95b1ef688d0df6209276a5fc71a082174f4a27f38c103", 16)
c3 = int("b8fe66dd64e0503393518db96e5f0c397247638b48f291b01c12f2548d13f169df309516c6ab992ec941f4306d5e53d4830850c5d9c13df16050b5f1fc7a253eab65ff4aded53b5099977db7441ca56830cfad5523e64b65409999b42ff12311f5ceffa3548fea020dbbbb9fcc76a37d128122ee4c6c6ae474fa1aed4643139372c9a7f552dadc29dc5475742826613d0d7293d0b44a9629d6aebe4e270278c5b774843c6901f03cc92e927df8f667f1179dd56a95eb1ecfba691dac4e83b153f24c25d997ab1757e77135fdf5786f11e742116f7726a3dea1bd930fb57cf97bcc299b3639c5a2419807796d5e1f635c35efa19cd062f82812b5e658187ff8cb", 16)

with open("keys/pkey_huangrobert.pem", 'rb') as f:
    public_key_pem = f.read()

# Importer la clé publique RSA
public_key = RSA.import_key(public_key_pem)

# Vérifier si la clé est une clé publique RSA
if isinstance(public_key, RsaKey) and not public_key.has_private():
    # Récupérer les composantes n et e
    n1 = public_key.n
    # print("Clé publique 1 importée avec n1 =", n1)

f.close()

with open("keys/pkey_ghoward.pem", 'rb') as f:
    public_key_pem = f.read()

# Importer la clé publique RSA
public_key = RSA.import_key(public_key_pem)

# Vérifier si la clé est une clé publique RSA
if isinstance(public_key, RsaKey) and not public_key.has_private():
    # Récupérer les composantes n et e
    n2= public_key.n
    # print("Clé publique 2 importée avec n2 =", n2)

f.close()

with open("keys/pkey_rebeccataylor.pem", 'rb') as f:
    public_key_pem = f.read()

# Importer la clé publique RSA
public_key = RSA.import_key(public_key_pem)

# Vérifier si la clé est une clé publique RSA
if isinstance(public_key, RsaKey) and not public_key.has_private():
    # Récupérer les composantes n et e
    n3 = public_key.n
    # print("Clé publique 3 importée avec n3 =", n3)

f.close()


# === CRT (Chinese Remainder Theorem) ===
def crt(c_list, n_list):
    N = n_list[0] * n_list[1] * n_list[2]
    result = 0
    for i in range(3):
        ni = n_list[i]
        ci = c_list[i]
        Ni = N // ni
        inv = inverse(Ni, ni)
        result += ci * inv * Ni
        # print(f"Calcul de CRT pour i={i}: ci={ci}, ni={ni}, Ni={Ni}, inv={inv}, result={result}")
    return result % N

# === Calcul de m³ ===
m_cubed = crt([c1, c2, c3], [n1, n2, n3])

# print("m_cubed =", m_cubed)

# === Racine cubique entière ===
m = gmpy2.iroot(m_cubed, 3)[0]

# print("m après racine cubique =", m)

# === Extraction du mot de passe depuis le padding PKCS#1 ===
plaintext = long_to_bytes(m)
# print("Plaintext en bytes =", plaintext)

# L'encodage PKCS#1 v1.5 commence par 0x00 0x02 [padding] 0x00 [message]
password = plaintext.split(b'\x00')[-1]

# Affichage du mot de passe
print({password.decode(errors='ignore')})


{'mot de passe : 32a88d8318744e2c3cad3e83c8e8a628'}


# FLAG 17 : 
rsa.shared Atrium RC 25

On récupère d'abord les clés publiques de megan77 et dylannewman. Les 2 utlisateurs partagent le même N donc en utilisant le théorème de Bézout et le théorème des restes chinois on peut trouver m.

Attention le code du digicode se situe à la fin du message déchiffré.


In [1]:
from pathlib import Path
from sympy import mod_inverse
from cryptography.hazmat.primitives.serialization import load_pem_public_key

def hex_vers_int(hex_str):
    return int(hex_str, 16)

# === Chargement des messages chiffrés ===
chiffre_megan = hex_vers_int("00a7f083459072de4291d6f2adaff243aa5fca351fd70f0d4b59aa1ee7c868b11d4e2bf14bd42d64453503386f53342793a8abd03882c83e27e32ab6f374b4524fc17438cc189a3c0b14eda38b2b807005ac7a2128e85d634602f189b21e4bd74afd90bb09cc364a2fdf911346bc6cc12cd91ad0615e4c67d1f2487a925f9cc951334a0f234e655077e2ef97c1706868fc7e0aee3240c6344c1ce7cc0fadec9e85124cb18129a496989640da0d2c2561c80a02818955536bc2955bc45b4551e122be7421b76eb7af42b144459fa621201a78d821e9b6700ab11c2f6c78f3e63fd2f5a25a8cb19db6ef5b4e6b4fc0fef2506f373280f4f66c5d60e9e10866b75f")

chiffre_dylan = hex_vers_int("6d0bc4a8381042458eaa7e485c7b12ee9e5a1a3dbd94eb6d42bfa56734012fa77aa97ff0474d1067eed1f3e753818f2e9b0d7068be18678529f127bac6a35b74d41e12c4125f4a7288fff5ccb9829ba135b4baaae8bca1ace8e60dc1865993240739daebe2793a3f6242c437ed6a9686d79f614b7754a92ba6b779d892f830a99b666beec32e48702da13fa3988db3a0d3863e73ec78a0a7b31d483781d7f2a11d1cc236b268a383f46874303db8216253272abfc8a5bda11868a93e54f999c3cad3ef24421989ff994ade5a0e6ef1c6fcb189cef5672b1d80b5bd36cec5b13f31507624c06452af454d714b7e32366a2de40b3a9058a6ec260cfc0d1584a678")


def charger_cle_publique(chemin_fichier):
    contenu = Path(chemin_fichier).read_bytes()
    cle = load_pem_public_key(contenu)
    infos = cle.public_numbers()
    return infos.e, infos.n

def algorithme_euclide_etendu(a, b):
    if a == 0:
        return b, 0, 1
    d, x, y = algorithme_euclide_etendu(b % a, a)
    return d, y - (b // a) * x, x

def dechiffrer_message_crt(cipher1, cipher2, exp1, exp2, mod):
    _, u, v = algorithme_euclide_etendu(exp1, exp2)
    part1 = pow(cipher1, u, mod)
    part2 = pow(cipher2, v, mod)
    message = (part1 * part2) % mod
    return message.to_bytes((message.bit_length() + 7) // 8, 'big')



# === Chargement des clés publiques ===
exp_megan, mod_megan = charger_cle_publique("keys/pkey_megan77.pem")
exp_dylan, mod_dylan = charger_cle_publique("keys/pkey_dylannewman.pem")

# Vérification des modules
if mod_megan != mod_dylan:
    raise ValueError("Les modules RSA ne sont pas identiques. CRT non applicable.")

# === Déchiffrement ===
message_clair = dechiffrer_message_crt(
    chiffre_megan, chiffre_dylan, exp_megan, exp_dylan, mod_megan
)

print("Message déchiffré :", message_clair)


Message déchiffré : b'\x02\x18\xfb\xf8\x81\xe8\xe9\xfc\xbaD\x91\rm\xc4\x90\xce\xcc\xfc\xb5>oT}\xe99\x8c\xa9`\xe6\x94\xc8h%\xdb\x087\x1e\xd6\xbd}\x10\xd6\xc8bCN\xb9D}\xe7\x7f\x89\xed\xf2\x89\xf6\xd3\x02h\xc37r\xa0o\x86Iw\xcb\xe5`\xe0\xb9\x03s\xf4\x0ca;\xec\x05U\x93\xf1uN&\xd6E\xdd\xc8\xca\xed\xe8*\xa4\x7fj\xaa|\x01+\xd9\xf9\x84\xbe\xaf\x11\xeb\x83\x9c>\xe2\xd2\xbb\xa4\xc3Z\x98\x97\x81gP\xcd\x0e\xc68\xe5\xfff\xeb7\xfe\xf1\x89\x1b\xb8d\x9c\xa3h\xc9\xa6\xc2@i>&\x9er\xac7\x0e\x87\x86\xad\x1c/.\x93TSc{\xd1#S\x8eCA\xac\x1e\xe0yr\x1d\xdat\xee.\xb7\x92x\x97e\xb7\x00Nouveau digicode de la salle RC-25 : 1631ffe37471b8b27b9237a76bcf6179'


# FLAG 18 : 
secret.sharing Firmware

voir code sage "flag18.sage"

Pour effectuer ce flag il faut d'abord se rendre en RC-25 à l'atrium. On copie depuis le terminal tous les MAC de nos 9 premiers flags. Ensuite, on les utilise pour extraire les paires (X,A) qui permettent, via une interpolation de Lagrange, de reconstruire le polynôme secret R(x). En évaluant ce polynôme en 0, on obtient la valeur R(0) qui est la clé de mise à jour du firmware.

Enfin, une fois qu'on a le secret partagé (qui correspond à la firmware key), on se rend au CICSU et on entre la clé dans la borne de mise à jour de firmware.

Remarque : il y'a une autre firmware key qui traine en RC-07 

# FLAG 19 
 credit.fraud (WEB SERVICE CICSU)

Dans le terminal du webservice du CICSU (option 3), on peut récupérer un 'batch' de plusieurs transactions bancaires. Le but étant de vérifier si chaque transactions est frauduleurse ou pas. Pour cela, on vient vérifier les quatre points suivants (voir docu à la bibliothèque) : 
- La validité de la signature RSA produite par la carte.
- L’authenticité de la clé publique de la banque, via le certificat (la clé publique est dans un des certificats émis par la banque)
- La validité du certificat de la carte, émis par une banque 
- L’authenticité du certificat de la banque, signé par l’autorité centrale (en utilisant le certificat de l'autorité centrale : "certificat/certificat_cicsu.pem")

Enfin, le code utilisé renvoie une liste de 0 et de 1 (associé à chaque transaction) :
- vaut 0 si la transaction est frauduleuse
- 1 sinon

Pour valider le flag, il faut copier coller la liste de 0/1 dans l'option 4 du terminal de webservice.



In [2]:
import yaml
import json
import subprocess
import re

def verifier_transaction(transaction):
    resultats = []

    # Vérifier si le CN du certificat correspond au numéro de carte
    num_carte = transaction.get("card", {}).get("number", "")
    num_carte = num_carte.replace(" ", "").replace("-", "")
    # print("num_carte =", num_carte)  

    cert_carte = transaction.get("card", {}).get("certificate", "")
    cert_banque = transaction.get("card", {}).get("bank", {}).get("certificate", "")

    # print("cert_carte =", cert_carte)  
    with open("cert_carte.pem", "w") as f:
        f.write(cert_carte.strip())

    process = subprocess.run(["openssl", "x509", "-noout", "-subject"], input=cert_carte, capture_output=True, text=True)
    ligne_subject = process.stdout.strip()
    # print("ligne_subject =", ligne_subject)  

    match = re.search(r"CN\s*=\s*([^,\n]+)", ligne_subject)
    if match:
        cn = match.group(1)
        cn = cn.replace(" ", "").replace("-", "")
    else:
        cn = ""
    # print("cn =", cn)  

    if cn == num_carte:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si la banque est une autorité de certification (CA:TRUE)
    process = subprocess.run(["openssl", "x509", "-noout", "-text"], input=cert_banque, capture_output=True, text=True)
    texte_cert = process.stdout
    # print("texte_cert =", texte_cert)  

    if "CA:TRUE" in texte_cert:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si le certificat de la carte est signé par la banque
    nom_banque_data = json.loads(transaction.get("data", "{}")).get("bank-name", "")
    # print("nom_banque_data =", nom_banque_data)  

    process = subprocess.run(["openssl", "x509", "-noout", "-issuer"], input=cert_carte, capture_output=True, text=True)
    issuer_carte = process.stdout
    # print("issuer_carte =", issuer_carte)  

    if nom_banque_data in issuer_carte:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si le certificat de la banque est signé par l'autorité
    process = subprocess.run(["openssl", "x509", "-noout", "-issuer"], input=cert_banque, capture_output=True, text=True)
    issuer_banque = process.stdout.strip()
    # print("issuer_banque =", issuer_banque)  

    with open("certificat/certificat_cicsu.pem", "r") as f:
        cert_autorite = f.read()

    process = subprocess.run(["openssl", "x509", "-noout", "-issuer"], input=cert_autorite, capture_output=True, text=True)
    issuer_autorite = process.stdout.strip()
    # print("issuer_autorite =", issuer_autorite)  

    if issuer_autorite in issuer_banque:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si le nom de la banque dans la transaction correspond à celui de la carte
    nom_banque_carte = transaction.get("card", {}).get("bank", {}).get("name", "")
    # print("nom_banque_carte =", nom_banque_carte)  

    if nom_banque_data == nom_banque_carte:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier la validité du certificat de la carte (signé par la banque)
    with open("cert_banque.pem", "w") as f:
        f.write(cert_banque.strip())

    cmd = ["openssl", "verify", "-trusted", "certificat/certificat_cicsu.pem", "-untrusted", "cert_banque.pem", "cert_carte.pem"]
    process = subprocess.run(cmd, capture_output=True, text=True)
    # print("process.stdout =", process.stdout)  

    if "OK" in process.stdout:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si la signature du challenge est correcte
    challenge = transaction.get("data", "")
    signature = transaction.get("signature", "")
    signature_binaire = bytes.fromhex(signature)

    with open("challenge.txt", "w") as f:
        f.write(challenge.strip())

    with open("signature.der", "wb") as f:
        f.write(signature_binaire)

    subprocess.run(["openssl", "x509", "-in", "cert_carte.pem", "-noout", "-pubkey", "-out", "pubkey.pem"], capture_output=True)

    process = subprocess.run(["openssl", "dgst", "-sha256", "-verify", "pubkey.pem", "-signature", "signature.der", "challenge.txt"],
                             capture_output=True, text=True)
    # print("process.stdout =", process.stdout)  

    if "Verified OK" in process.stdout:
        resultats.append(1)
    else:
        resultats.append(0)

    # Vérifier si le certificat de la banque est auto-signé
    process = subprocess.run(["openssl", "x509", "-noout", "-issuer"], input=cert_banque, capture_output=True, text=True)
    issuer = process.stdout.strip()
    # print("issuer =", issuer)  

    if "CN=__fake__" in issuer:
        resultats.append(0)
    else:
        resultats.append(1)

    # print("resultats =", resultats)  

    if 0 in resultats:
        return 0
    else:
        return 1


with open("batch.yaml", "r") as f:
    donnees = yaml.safe_load(f)

resultats_transactions = []

for transaction in donnees.get("batch", {}).get("transactions", []):
    # print("transaction =", transaction)  
    resultat = verifier_transaction(transaction)
    resultats_transactions.append(str(resultat))

print(",".join(resultats_transactions))


1,0,1,1,0,0,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,1,1,1,0,0,1,0,1,0,0,0,0,0,1,1


# FLAG20 : 
multi.lamport Catalogue, signature Lamport

Attention il faut se connecter avec la sécurité _ _pk_ _ au début du jeu

On copie 10 fois le résultat du doc 24 de la bibliothèque dans json

On tape admin dans le catalogue et on signe le message qu'on nous donne : "ships sebum ceder widen unlit"


In [3]:
import json
from hashlib import sha256
from binascii import unhexlify

# message à Signer
msg = b"levee poach gists shell crept"

class LamportPrivateKey:
    def __init__(self, elements_cle):
        self.cle = elements_cle
    
    def signer(self, msg):
        sig = b''
        h = int.from_bytes(sha256(msg).digest(), byteorder='big')
        # print("Hash initial (h) =", h) 
        for i in range(256):
            b = (h >> i) & 1
            # print(f"i={i}, b={b}") 
            sig += self.cle[2 * i + b]
        return sig.hex()

def reconstruire_composants(msgs, signes):
    composants = {}
    for msg, signe in zip(msgs, signes):
        signature_bytes = unhexlify(signe)
        # print("signature_bytes =", signature_bytes) 
        hash_digest = sha256(msg).digest()
        # print("hash_digest =", hash_digest) 
        h = int.from_bytes(hash_digest, byteorder='big')
        # print("Hash après conversion (h) =", h) 
        for i in range(256):
            bit = (h >> i) & 1
            index = 2 * i + bit
            # print(f"i={i}, bit={bit}, index={index}") 
            composants[index] = signature_bytes[i*32:(i+1)*32]
            # print(f"composants[{index}] = {composants[index]}") 
    return composants

chemin_fichier = 'catalogue.json'
messages = []
signatures = []

with open(chemin_fichier, 'r') as f:
    data_array = json.load(f)
    # print("data_array =", data_array) 
    for data in data_array:
        messages.append(data["message"].encode())  
        signatures.append(data["signature"])

# print("messages =", messages) 
# print("signatures =", signatures) 
composants = reconstruire_composants(messages, signatures)

elements_cle_privee = [b''] * 512
# print("elements_cle_privee (avant remplissage) =", elements_cle_privee) 
for index, composant in composants.items():
    elements_cle_privee[index] = composant
# print("elements_cle_privee (après remplissage) =", elements_cle_privee) 

lamport_cle = LamportPrivateKey(elements_cle_privee)

print("Signature :", lamport_cle.signer(msg))


Signature : 4a3ecdb49b895c76cebe895c0e6aa5d50e8fa6ff05c7430adfd848caf7bf4bb37c49e93a4844a216861b9b585e65fa6f5c80aab1ad95f06bc9acf82625d99e8de6b81008af8dad6f18b5355c77266611380b4b47412c59c1393b69b4de80cbc04f4861a53d3ebe706d50efe68cb076d1b6671cb1374c0431c33f6b892fd382512fb08c46c5f1992217ea6d427afe5dfb4b8404486929b8b3a9f802a1c980776dad508bc17a9d135b595d40ef089ef75913eb0a52c12016e05c7f8858784935039005173cca6e88155f8c418d1a90056f167cdf8d5d9fe58bd34c536c2d375b6c88f7e2ac161917b553cbd5b956eeed8522a11957f1609fc84bb6952cb7ee214df075b075d624c8a98410a8b49a0fa6aed062a60addd71031a86541de201f1e9d1794b83a5d361119c8f2a56b8f45371b90ba932c0b52324444202b0c34c8709843803d5aa9f087f02a7b2568a253d99d3cecaa5a80e26a1a37d114e59d7662e1ea29ac5a402a38eca8f1d0da61f7bd979fe50bd4a2ebc03196ba789dede6bb54203b2fb0b9661c74bd818f750349999a53f774d78fa12a7610a785a29466f0c0b24fcddb5d095a5e25ab99b0920c05fd0378e79af0f561d8e95f5fa82f134e7c2559c5ca350256eb1adc1db66b762d3291bb2de25fd8c09bfd8f27e11d814fdf315279c475d5d1c38a5a3e6799b6